In [1]:
import os
from typing import Optional, Type
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain.tools import BaseTool
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Set up your API key
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Define the input schema for the ReflectionInsightTool
class ReflectionInsightInput(BaseModel):
    reflection: str = Field(..., description="User's reflection with questions and answers.")

# Define the tool for generating insights
class ReflectionInsightTool(BaseTool):
    name: str = "generate_insight"
    description: str = "Generates insights based on user reflections about questions and answers."
    args_schema: Optional[Type[BaseModel]] = ReflectionInsightInput
    agent_executor: Optional[AgentExecutor] = None

    def _run(self, reflection: str) -> str:
        if self.agent_executor is None:
            raise ValueError("Agent executor is not set.")
        return self.agent_executor.invoke({"reflection": reflection})["output"]

    async def _arun(self, reflection: str) -> str:
        return self._run(reflection)

    def set_agent_executor(self, agent_executor: AgentExecutor):
        self.agent_executor = agent_executor

# Create the Gemini model
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7,
    google_api_key = os.environ.get("GOOGLE_API_KEY")
)

# Initialize the ReflectionInsightTool
reflection_tool = ReflectionInsightTool()

# Create the tools list
tools = [reflection_tool]

# Define the prompt template
template = """
You are an insightful reflection analyst. Your job is to provide deep insights based on a user's reflection,
which includes questions they've been asked and their answers.

When analyzing reflections, consider:
1. Patterns in the user's thinking
2. Potential blind spots or biases
3. Areas for further exploration or growth
4. Connections between different answers
5. Underlying themes or values revealed

Provide thoughtful, nuanced insights that help the user gain new perspectives on their reflections.

Available tools:
{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

User Reflection:
{reflection}

To solve this problem, think through the following steps:

1. Understand the overall context of the reflection
2. Identify key themes and patterns
3. Consider underlying motivations or beliefs
4. Formulate insights that provide value to the user
5. Present your insights in a clear, organized manner

Use the generate_insight tool to formulate your insights.

{agent_scratchpad}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["reflection", "agent_scratchpad", "tool_names", "tools"]
)

# Create the agent
agent = create_react_agent(
    llm=model,
    tools=tools,
    prompt=prompt
)

# Create the agent executor
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3,
)

# Set the agent executor to the tool
reflection_tool.set_agent_executor(agent_executor)

# Function to get insights from user reflection
def get_insights(reflection: str) -> str:
    response = agent_executor.invoke({"reflection": reflection})
    return response["output"]

# Example usage
if __name__ == "__main__":
    sample_reflection = """
    Question 1: What are your career goals for the next 5 years?
    Answer: I want to transition from a developer role to a team lead position. I enjoy mentoring others and want to have more impact on project direction.

    Question 2: What skills do you think you need to develop to achieve these goals?
    Answer: I need to improve my communication skills and learn more about project management. Technical skills are important, but I think soft skills will be more crucial.

    Question 3: What's holding you back from pursuing these goals?
    Answer: I sometimes doubt if I have the natural leadership qualities needed. I'm also comfortable in my current role and worried about the added stress.
    """

    insights = get_insights(sample_reflection)
    print(insights)

/home/user/task-prioritization/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: Okay, I've reviewed the user's reflection. Here's what I'm seeing:

*   **Theme:** The user wants to move into a team lead role within the next 5 years.
*   **Skills Gap:** They recognize the need to develop communication and project management skills, emphasizing the importance of soft skills.
*   **Obstacles:** Self-doubt about leadership qualities and comfort in their current role are holding them back. Fear of added stress is also a factor.

Now, let's use the `generate_insight` tool to get some deeper insights based on these observations.

Action: generate_insight
Action Input: ```json
{
  "reflection": {
    "question1": "What are your career goals for the next 5 years?",
    "answer1": "I want to transition from a developer role to a team lead position. I enjoy mentoring others and want to have more impact on project direction.",
    "question2": "What skills do you think you need to develop to achieve these goals?",
    "answer2": "I need to improve my communication sk

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: Okay, I need to analyze the user's reflection on their career goals. The reflection includes their goals, the skills they need, and what's holding them back. I should look for patterns, potential blind spots, and areas for growth, and then use the `generate_insight` tool to create the insights.

Action: generate_insight
Action Input:
```json
{
  "reflection": {
    "question1": "What are your career goals for the next 5 years?",
    "answer1": "I want to transition from a developer role to a team lead position. I enjoy mentoring others and want to have more impact on project direction.",
    "question2": "What skills do you think you need to develop to achieve these goals?",
    "answer2": "I need to improve my communication skills and learn more about project management. Technical skills are important, but I think soft skills will be more crucial.",
    "question3": "What's holding you back from pursuing these goals?",
    "answer3": "I sometimes doubt if I have the natural l

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


...

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Thought: The reflection provides insights into the user's career goals, skills development, and potential obstacles. I need to analyze the reflection to identify patterns in the user's thinking, potential blind spots, areas for further exploration, connections between different answers, and underlying themes or values revealed. Then, I will use the `generate_insight` tool to formulate insights that provide value to the user.

Action: generate_insight
Action Input: {
  "reflection": {
    "question1": "What are your career goals for the next 5 years?",
    "answer1": "I want to transition from a developer role to a team lead position. I enjoy mentoring others and want to have more impact on project direction.",
    "question2": "What skills do you think you need to develop to achieve these goals?",
    "answer2": "I need to improve my communication skills and learn more about project management. Technical skills are important, but I think soft skills will be more crucial.",
    "questio

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


Thought: I need to analyze the user's reflection to identify patterns, blind spots, and areas for growth. The user wants to move into a team lead position. They recognize the need for improved communication and project management skills. The user also expresses doubt about their leadership qualities and comfort in their current role.

I will use the `generate_insight` tool to formulate insights based on these observations.

Action: generate_insight
Action Input: {
  "reflection": {
    "question1": "What are your career goals for the next 5 years?",
    "answer1": "I want to transition from a developer role to a team lead position. I enjoy mentoring others and want to have more impact on project direction.",
    "question2": "What skills do you think you need to develop to achieve these goals?",
    "answer2": "I need to improve my communication skills and learn more about project management. Technical skills are important, but I think soft skills will be more crucial.",
    "question3

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).